In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "musamrc/model_256_256", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
{}

Input:
{}

Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
from transformers import TextStreamer, AutoTokenizer, AutoModelForCausalLM
import torch
import openpyxl

# TextStreamer nesnesini oluşturun
text_streamer = TextStreamer(tokenizer)

# Excel dosyasını yükleyin
wb = openpyxl.load_workbook('excell_step3.xlsx')
sheet = wb.active

# Excel dosyasındaki verileri kullanarak modeli çalıştırın
for row in range(2, sheet.max_row + 1):
    instruction = sheet[f'B{row}'].value
    input_text = sheet[f'C{row}'].value

    # Model için girdileri hazırlayın
    inputs = tokenizer(
        [
            alpaca_prompt.format(instruction, input_text, "")
        ],
        return_tensors="pt"
    ).to("cuda")

    # Modeli çalıştırın ve sonuçları alın
    outputs = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

    # Sonuçları decode edin ve Excel dosyasına yazın
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # İlk olarak, sadece modelin ürettiği çıktıyı (output) ayıklayın
    output_only = result.split(input_text)[-1].strip()
    final_output = output_only.replace('Response:', '').strip()

    # Son olarak, temizlenmiş çıktıyı Excel dosyasının F sütununa yazdırın
    sheet[f'F{row}'].value = final_output



# Excel dosyasını kaydedin
wb.save('dosyanızın_yolu.xlsx')